![Hand with calculator](calculator.jpg)

Did you know that the average return from investing in stocks is [10% per year](https://www.nerdwallet.com/article/investing/average-stock-market-return) (not accounting for inflation)? But who wants to be average?! 

You have been asked to support an investment firm by analyzing trends in high-growth companies. They are interested in understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. Providing them with this information gives them a competitive insight as to industry trends and how they should structure their portfolio looking forward.

You have been given access to their `unicorns` database, which contains the following tables:

## dates
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `date_joined` | The date that the company became a unicorn.  |
| `year_founded` | The year that the company was founded.       |

## funding
| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| `company_id`       | A unique ID for the company.                 |
| `valuation`        | Company value in US dollars.                 |
| `funding`          | The amount of funding raised in US dollars.  |
| `select_investors` | A list of key investors in the company.      |

## industries
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `industry`     | The industry that the company operates in.   |

## companies
| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| `company_id`   | A unique ID for the company.                      |
| `company`      | The name of the company.                          |
| `city`         | The city where the company is headquartered.      |
| `country`      | The country where the company is headquartered.   |
| `continent`    | The continent where the company is headquartered. |


# The output

Your query should return a table in the following format:
| industry  | year | num\_unicorns       | average\_valuation\_billions |
| --------- | ---- | ------------------- | ---------------------------- |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |

Where `industry1`, `industry2`, and `industry3` are the three top-performing industries.

In [2]:
import pandas as pd
import pandasql
import duckdb

In [4]:
companies = pd.read_csv('companies.csv')
dates = pd.read_csv('dates.csv')
funding = pd.read_csv('funding.csv')
industries = pd.read_csv('industries.csv')

In [8]:
result = duckdb.query("""
WITH top_industries AS (
  SELECT i.industry
  FROM industries i
  INNER JOIN dates d ON i.company_id = d.company_id
  WHERE EXTRACT(YEAR FROM d.date_joined::DATE) IN (2019, 2020, 2021)
  GROUP BY i.industry
  ORDER BY COUNT(*) DESC
  LIMIT 3
),
yearly_stats AS (
  SELECT 
    ti.industry,
    EXTRACT(YEAR FROM d.date_joined::DATE) AS year,
    COUNT(*) AS num_unicorns,
    AVG(f.valuation) AS avg_valuation
  FROM top_industries ti
  INNER JOIN industries i ON ti.industry = i.industry
  INNER JOIN dates d ON i.company_id = d.company_id
  INNER JOIN funding f ON d.company_id = f.company_id
  WHERE EXTRACT(YEAR FROM d.date_joined::DATE) BETWEEN 2019 AND 2021
  GROUP BY ti.industry, EXTRACT(YEAR FROM d.date_joined::DATE)
)
SELECT 
  industry,
  year,
  num_unicorns,
  ROUND(avg_valuation / 1000000000, 2) AS average_valuation_billions
FROM yearly_stats
ORDER BY year DESC, num_unicorns DESC;
""").to_df()

result

,industry,year,num_unicorns,average_valuation_billions
0,Fintech,2021,138,2.75
1,Internet software & services,2021,119,2.15
2,E-commerce & direct-to-consumer,2021,47,2.47
3,Internet software & services,2020,20,4.35
4,E-commerce & direct-to-consumer,2020,16,4.00
5,Fintech,2020,15,4.33
6,Fintech,2019,20,6.80
7,Internet software & services,2019,13,4.23
8,E-commerce & direct-to-consumer,2019,12,2.58


In [10]:
result.to_csv('unicorns.csv', index=False)